# Calculate the feedbacks in single model abrupt-4xCO2 experiment by using the radiatvie kernel 

>ref: [Soden, et.al., (2008)](https://doi.org/10.1175/2007JCLI2110.1)


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import time
# import the function as r3k 
import Radiative_Repsonse_with_Raditive_kernel as r3k


In [2]:
# this is for example data
# create your own function for your data
def read_postprocessed_data_example():
    var_list = 'ta hus ts rlut rsdt rsut rlutcs rsutcs rsus rsds'.split()
    # control data in dictionary {var: xarray,...}
    # Coordinate: [month,plev,lat,lon] month.size=12
    var_cont = {}
    for var in var_list:
        filepath = f"./data/piControl/GFDL-CM4/{var}.mon.0001-0200.ltm.nc.r1i1p1f1.2x2.5.2021052021"
        var_cont[var] =  xr.open_dataset(filepath)[var].isel(model=0).load() # load into memory
    # perturbation data in dictionary {var: xarray,...}
    # Coordinate: [time,plev,lat,lon], time.size=yearx12
    var_pert = {}
    for var in var_list:
        filepath = f"./data/abrupt-4xCO2/GFDL-CM4/{var}.mon.0001-0150.nc.r1i1p1f1.2x2.5.2021052021"
        var_pert[var] =  xr.open_dataset(filepath)[var].isel(model=0).load() # load into memory
    return var_cont, var_pert 
# read kernel data
def read_kernel_file(rk_source='GFDL'):
    rkpath = "./data/kernels_TOA_"+rk_source+"_CMIP6-standard.nc"
    f_RK =  xr.open_dataset(rkpath,decode_times=False) 
    if rk_source =='GFDL':
        f_RK =  f_RK.rename({'time': 'month'})
        f_RK.coords['month'] = np.arange(1,13,1)
    return f_RK

In [3]:
%%time
# read data
f_RK =  read_kernel_file()
var_cont, var_pert = read_postprocessed_data_example()

CPU times: user 140 ms, sys: 1.71 s, total: 1.85 s
Wall time: 2.06 s


In [4]:
%%time
# compute RR with RK
ds_rk_numba =  r3k.decompose_dR_rk_toa_core(var_pert,var_cont,f_RK )

CPU times: user 1min 33s, sys: 4.45 s, total: 1min 38s
Wall time: 13.6 s


In [5]:
%%time
import Radiative_Repsonse_with_Raditive_kernel_numpy as r3k_np
ds_rk_np =  r3k_np.decompose_dR_rk_toa_core_numpy(var_pert,var_cont,f_RK )

/tigress/cw55/work/rk_example/Radiative_Repsonse_with_Raditive_kernel_numpy.py:233: RuntimeWarning: invalid value encountered in true_divide
  alb_cont_mon = cont_rsus[mon,:,:] / cont_rsds[mon,:,:]
/tigress/cw55/work/rk_example/Radiative_Repsonse_with_Raditive_kernel_numpy.py:237: RuntimeWarning: invalid value encountered in true_divide
  alb_pert_mon = pert_rsus[i*12+mon,:,:] / pert_rsds[i*12+mon,:,:]


CPU times: user 27.8 s, sys: 2.96 s, total: 30.8 s
Wall time: 30.9 s


In [6]:
%%time
import Radiative_Repsonse_with_Raditive_kernel_xarray as r3k_xr
ds_rk_xr=  r3k_xr.decompose_dR_rk_toa_core_xarray(var_pert,var_cont,f_RK )

CPU times: user 45.9 s, sys: 38.1 s, total: 1min 24s
Wall time: 1min 24s


In [7]:
# 0.1% error check
xr.testing.assert_allclose(ds_rk_numba,ds_rk_np,rtol=1e-3,atol=1e-3)
xr.testing.assert_allclose(ds_rk_numba,ds_rk_xr,rtol=1e-3,atol=1e-3)